In [1]:
using LinearAlgebra, Statistics
using LaTeXStrings, Plots, Interpolations, NLsolve, Optim, Random, Parameters
using Optim: maximum, maximizer

# Stochastic Optimal Growth Model

### The problem

Let the sequence of shock process $\{\xi_i\}$ to be i.i.d., with the common distribution denoted by $\phi$.

The household's problem is to maximize
$$ \mathbb{E}_0 \sum_{t=0}^{\infty} \beta^t u(c_t) $$
subject to 
$$ y_{t+1} = f(y_t - c_t) \xi_{t+1}, \; 0\leq c_t \leq y_t $$

Note: one assumes full depreciation of capital each period.


### Recursive formulation

The problem can be casted in the following recurisve formulation:
$$ V(y) = \max_{0\leq c\leq y} \; u(c) + \mathbb{E} \beta V(f(y - c) \xi) $$

## Pseudo-codes
We assume a lognormal distribution of the income shock. A simple way to evaluate the integral is by performing a Monte Carlo draw and evaluate the mean.

### Method 1: Fitted value function iteration

> 1. Choose grid points $\{y_1, ... y_n\}$. Choose an initial value function $\{v_1, ... v_n\}$. Use linear interpolation to evaluate the value function at points between the two grid points.
> 2. For each $1\leq i \leq n$, maximize the RHS using a non-linear optimizer. Keep track of both the optimal value attained as well as the optimal policy $c_i$. Update $v_i$ with the attained optimal value.
> 3. Iterate until convergence as specified by some threshold.

### Method 2: Time iteration/Policy function iteration (Coleman 1990)


### Method 3: Endogenous grid method (Carroll 2006)

In [25]:
function T(w; p, tol = 1e-10)
    (; β, u, f, ξ, y) = p
    w_func = LinearInterpolation(y, w)

    Tw = similar(w)
    σ  = similar(w)

    for (i, y_val) in enumerate(y)
        results = maximize(c -> u(c;p) + β * mean(w_func.(f(y_val - c;p) .* ξ)), tol, y_val)
        Tw[i] = maximum(results)
        σ[i]  = maximizer(results)
    end
    return (;w = Tw, σ)
end
    

6-element extrapolate(scale(interpolate(::Vector{Float64}, BSpline(Linear())), (0.0:0.2:1.0,)), Throw()) with element type Float64:
 4.5
 3.5597036591092523
 0.3939459310451876
 1.5610820754199866
 1.6958202377275753
 5.410947928995602

In [ ]:
Random.seed!(42)

u(c;p) = log(c) # utility
f(k;p) = k^p.α # deterministic part of production function

OptimalGrowthModel = @with_kw (β = 0.96, u = u, f=f, ξ=, y)

In [49]:
exp.(0 .+ 0.1 * randn(250))

250-element Vector{Float64}:
 0.9556013519374775
 1.1934980177813883
 1.172402625244654
 0.8807964231677283
 1.025425213262734
 1.1219033904445948
 0.9256282002401406
 0.8679126375108249
 1.129968329092908
 0.9497501526704041
 ⋮
 0.8986317974474225
 0.9112953714029784
 0.9256661376656466
 1.0210821636775953
 0.8508324038199886
 1.0485668635767804
 0.9342587903560345
 1.0167332989834477
 0.9924991587807761

In [42]:
u(c;p) = log(c) # utility
f(k;p) = k^p.α # deterministic part of production function

f (generic function with 1 method)